**Imports**

In [37]:
import urllib2
import re
import operator
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib

In [38]:
#Get all the locations in game of thrones
response = urllib2.urlopen('http://gameofthrones.wikia.com/wiki/Category:Locations')
html = response.read()
linkList = (re.findall(r'\<a href=\"/wiki/(.*?)\"\>',html))

In [39]:
#Functions helping to clean up html code
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

In [40]:
baseurl = 'http://gameofthrones.wikia.com/wiki/'
placeDict = {}

#Looking up each of the locations
for link in linkList:
  html = urllib.urlopen(baseurl + link.split("\"")[0] + '?action=edit').read()
  text = (text_from_html(html))
  
  #Defining where the target text starts and ends
  start = text.find('\'\'\'')
  end = text.find('==See also==')
  
  #Trimming the text to only get the relevant part
  mainText = text[start:end]
  mainText = re.sub('[\[\]\'=\\n]', '', mainText)
  mainText = re.sub('<ref>.*?</ref>', '', mainText)


  placeDict[link] = [x.lower() for x in mainText.split()]


In [41]:
# open dataset with sentiment and parse the file and save it into 
# dictiotionary in form {Word: sentiment}
file = open('dataset1.TXT', 'r')

i = 1
senWordLitsDic = {}
for line in file:
    listLine = line.split()
    if i>4:
        senWordLitsDic[listLine[0]] = float(listLine[2])
    i += 1

# Get list of the words and calculate average sentiment using dic
def calcAvgSentiment(tokenList, dic = senWordLitsDic):
    suma = 0
    nrSentimentWords = 0
    for word in tokenList:
        lword = word.lower()
        # Calculate sentiment only for words 
        if lword in dic:
            suma += dic[lword]
            nrSentimentWords +=1
    if nrSentimentWords == 0:
        print "no word has sentiment"
        return 0 
    return suma/nrSentimentWords

In [42]:
#Calculating the average sentiment for all the locations
sentimentDict = {}
for link, val in placeDict.items():
    sentimentDict[link] = calcAvgSentiment(val)

no word has sentiment
no word has sentiment
no word has sentiment
no word has sentiment
no word has sentiment
no word has sentiment


In [44]:
sorted_sentiment = sorted(sentimentDict.items(), key=operator.itemgetter(1))
print 'The 5 locations with highest sentiment: \n [%s] \n' % '\n '.join(map(str, reversed(sorted_sentiment[-5:])))

The 5 locations with highest sentiment: 
 [('Palace_of_Justice" title="Palace of Justice', 5.625777777777778)
 ('Mother_of_Mountains" title="Mother of Mountains', 5.588716216216221)
 ('Great_Wyk" title="Great Wyk', 5.536226415094341)
 ('Moon_Pool" title="Moon Pool', 5.512527472527474)
 ('Driftmark" title="Driftmark', 5.506666666666666)] 

